In [13]:
import numpy as np
import pandas as pd
import geopandas as gpd

#Next Task: Clean, Aggregate businesses#Read businesses:
bus_first = pd.read_csv("../raw_data/Business_Licenses_-_Current_Active.csv")


#Select only Chicago/Illinois businesses
bus = bus_first[bus_first.CITY=="CHICAGO"]

#Remove non-relevant columns
bus = bus[["LICENSE ID", "DOING BUSINESS AS NAME", 
           "ADDRESS", "CITY", "STATE", "ZIP CODE", 
          "LICENSE CODE", "LICENSE DESCRIPTION", "LOCATION"]]

bus["LICENSE DESCRIPTION"].value_counts()

#Create our own labels for categories:
#isin syntax help from here: https://stackoverflow.com/questions/18250298/how-to-check-if-a-value-is-in-the-list-in-selection-from-pandas-data-frame
#np structure help frum here: https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/
conditions = [
    (bus["LICENSE DESCRIPTION"].isin(["Limited Business License", 
                                   "Regulated Business License"])), 
    (bus["LICENSE DESCRIPTION"].isin(["Consumption on Premises - Incidental Activity", 
                                    "Tavern", "Late Hour"])), 
    (bus["LICENSE DESCRIPTION"] == "Retail Food Establishment"), 
    (bus["LICENSE DESCRIPTION"] == "Children's Services Facility License"), 
    (bus["LICENSE DESCRIPTION"].isin(["Public Place of Amusement", 
                                    "Music and Dance", 
                                    "Wrigley Field"]))
]
values = ["business", "bar", "restaurant", "daycare", "entertainment"]
bus["category"] = np.select(conditions, values)

#Select only businesses with those categories:
bus = bus[bus.category != "0"]

#Separate by geography
#notnull() idea: https://stackoverflow.com/questions/22551403/python-pandas-filtering-out-nan-from-a-data-selection-of-a-column-of-strings
bus_geog = bus[bus.LOCATION.notnull()]
bus_geog.head(2)

#Convert bus_geog to gpd

#Help from: https://www.geeksforgeeks.org/split-a-text-column-into-two-columns-in-pandas-dataframe/
#Help from: https://stackoverflow.com/questions/42349572/remove-first-x-number-of-characters-from-each-row-in-a-column-of-a-python-datafr
bus_geog
bus_geog[['Latitude','Longitude']] = bus_geog.LOCATION.str.split(",",expand=True)
bus_geog["Latitude"]= bus_geog["Latitude"].str[1:].astype("float")
bus_geog["Longitude"]= bus_geog["Longitude"].str[:-1].astype("float")
bus_geog = gpd.GeoDataFrame(bus_geog, 
                            geometry=gpd.points_from_xy(bus_geog.Longitude, 
                                                        bus_geog.Latitude))
bus_geog = bus_geog.set_crs("EPSG:3435")
bus_geog = bus_geog.to_crs("EPSG:3435")
bus_geog = bus_geog.to_crs("EPSG:4326")



<ipython-input-13-d461313ff31a>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus_geog["Latitude"]= bus_geog["Latitude"].str[1:].astype("float")
<ipython-input-13-d461313ff31a>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus_geog["Longitude"]= bus_geog["Longitude"].str[:-1].astype("float")


In [14]:
bus_geog = bus_geog[["LICENSE ID", "DOING BUSINESS AS NAME", "category", "geometry"]]

In [15]:
bus_geog.to_file("../intermediate_data/bus_geog.geojson")


<ipython-input-15-cf47314ed036>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  bus_geog.to_file("../intermediate_data/bus_geog.geojson")


In [6]:
bus_geog["category"].value_counts()

business         30122
restaurant       12845
bar               4112
daycare            965
entertainment      897
Name: category, dtype: int64